In [ ]:
# core LangChain 0.2+ components
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePick, RunnableParallel, RunnableLambda
from langchain.memory import ConversationBufferMemory
from langchain_ollama import OllamaLLM
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

# handy utilities
import os, pathlib
import dotenv
from langchain_community.document_loaders.pdf import PyPDFLoader

dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")  # or use dotenv / env vars

# ollama
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

ImportError: cannot import name 'SelfQueryRetriever' from 'langchain.retrievers.self_query' (/home/nitish/Documents/github/RAGStudio/.venv/lib/python3.12/site-packages/langchain/retrievers/self_query/__init__.py)

In [21]:
# Core LangChain interfaces (0.2+)
from langchain_community.chat_models import ChatOllama  # local chat LLM
from langchain_community.embeddings import OllamaEmbeddings  # local embeddings

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.faiss import FAISS

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePick, RunnableLambda

# Optional: LangSmith for tracing/eval
from langsmith import Client
import os, asyncio

### Basic RAG

In [11]:
# 1. ingest docs (10-Ks of JPMorgan as an example)
loader = PyPDFLoader("docs/ISLP_website.pdf")
docs_raw = loader.load()

# 2. split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=2750, chunk_overlap=600)
docs_split = splitter.split_documents(docs_raw)

Ignoring wrong pointing object 185 0 (offset 0)
Ignoring wrong pointing object 421 0 (offset 0)
Ignoring wrong pointing object 423 0 (offset 0)
Ignoring wrong pointing object 430 0 (offset 0)
Ignoring wrong pointing object 643 0 (offset 0)
Ignoring wrong pointing object 860 0 (offset 0)
Ignoring wrong pointing object 892 0 (offset 0)
Ignoring wrong pointing object 894 0 (offset 0)
Ignoring wrong pointing object 928 0 (offset 0)
Ignoring wrong pointing object 1047 0 (offset 0)
Ignoring wrong pointing object 1124 0 (offset 0)
Ignoring wrong pointing object 1138 0 (offset 0)
Ignoring wrong pointing object 1148 0 (offset 0)
Ignoring wrong pointing object 1173 0 (offset 0)
Ignoring wrong pointing object 1380 0 (offset 0)
Ignoring wrong pointing object 1405 0 (offset 0)
Ignoring wrong pointing object 1468 0 (offset 0)
Ignoring wrong pointing object 1520 0 (offset 0)
Ignoring wrong pointing object 1578 0 (offset 0)
Ignoring wrong pointing object 1670 0 (offset 0)
Ignoring wrong pointing objec

In [12]:
# 3. embed & store in FAISS
# embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
embeddings = OllamaEmbeddings(model="bge-m3:latest")
vectordb = FAISS.from_documents(docs_split, embeddings)

In [30]:
# 4. build a retriever
retriever = vectordb.as_retriever(search_kwargs={"k": 50})

# 5. template
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert data scientist who loves to teach. You are very verbose and provide long explanations. You are trevor hastie. Use ONLY the context to answer.",
        ),
        ("human", "Context: <context>\n{context}</context>\n\nQuestion: {question}"),
    ]
)

llm = ChatOllama(model="gemma3:12b")
chain = (
    RunnableParallel(context=retriever, question=RunnablePick(keys=["question"]))
    | template
    | llm
)

In [25]:
res = chain.invoke(
    {"question": "Explain linear regression in detail and its relation to LASSO"}
)

In [29]:
print(res.content)

Okay, let's break down linear regression and its relationship to LASSO (Least Absolute Shrinkage and Selection Operator). I'm pulling information from the provided text to do this.

**1. Linear Regression: The Basics**

*   **What it is:** Linear regression is a supervised learning method primarily used to predict a *quantitative* (continuous) response variable. Think of it as a way to model the relationship between one or more predictor variables and a target variable that can take on a range of values.
*   **Goal:**  The aim is to find the "best-fitting" line (or plane in multiple dimensions) that describes the relationship between the predictors and the response.
*   **Example:** Predicting sales based on advertising budget (TV, radio, newspaper).  The linear regression model would attempt to find an equation that best explains how changes in advertising spending affect sales.
*   **Why use it?**
    *   It's a foundational technique – many more complex statistical learning methods 

### Filter + Re-Ranker

In [31]:
base = vectordb.as_retriever()

# a) Self-Query: let GPT translate the user question into a vector search + metadata filter
self_q = SelfQueryRetriever.from_llm(
    llm, base, metadata_field_info=[("ticker", "str"), ("fiscal_year", "int")]
)

# b) Multi-Query: generate paraphrases of the question → union of results
multi_q = MultiQueryRetriever.from_llm(llm, base, num_queries=4)

# c) Re-rank: apply bge-reranker or Cohere-rerank
reranker = TopKReRank(k=4, model_name="bge-reranker-large")

# d) Compress: keep only the sentence fragments relevant to the query
cc_retriever = ContextualCompressionRetriever(
    base_retriever=multi_q, base_compressor=reranker
)

# use cc_retriever in the earlier chain
chain2 = (
    RunnableParallel(context=cc_retriever, question=RunnablePick(keys=["question"]))
    | template
    | llm
)

NameError: name 'SelfQueryRetriever' is not defined